In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from dl.dataset.datamodes.npz import rectifiedrgb, debayeredrgb
from dl.config.label_map_helper import LabelMapHelper, LabelConversion

import pandas as pd
import os
from pathlib import Path
import json
from tqdm import tqdm

In [2]:
output_dir_prod = '/mnt/sandbox1/alex.li/introspection/pmehta_2023_val_bestmodel/halo_rgb_stereo_test_v6_0'
output_dir_safety = '/mnt/sandbox1/alex.li/introspection/pmehta_2023_val_bestmodel/halo_humans_on_path_v3'
# output_dir_prod = '/mnt/sandbox1/alex.li/results/17902/halo_rgb_stereo_test_v6_0'
# output_dir_safety = '/mnt/sandbox1/alex.li/results/17902/halo_humans_on_path_v3'

In [3]:
with open(output_dir_prod + "/results.txt", 'r') as f:
    print(''.join(f.readlines()))


General metrics. Note TPs and FNs are based on ['Non-driveable', 'Trees_Weeds', 'Humans', 'Vehicles']:
true_positive: 10881
true_negative: 31499
false_positive: 855
false_negative: 961
precision_image: 0.9271472392638037
recall_image: 0.918848167539267
f1_image: 0.9229790482653321
productivity_image: 0.9735735921369846

General metrics on large objects:
large_object_true_positive: 0
large_object_false_negative: 0

Strict metrics on humans:
human_true_positive: 5454
human_true_negative: 37694
human_false_positive: 523
human_false_negative: 525
human_precision_image: 0.9124979086498243
human_recall_image: 0.9121926743602609
human_f1_image: 0.9123452659752426
human_productivity_image: 0.9863149907109402

Strict metrics on vehicles:
vehicle_true_positive: 1612
vehicle_true_negative: 41442
vehicle_false_positive: 1033
vehicle_false_negative: 109
vehicle_precision_image: 0.6094517958412098
vehicle_recall_image: 0.936664729808251
vehicle_f1_image: 0.7384333486028402
vehicle_productivity_imag

In [4]:
with open(output_dir_safety + "/results.txt", 'r') as f:
    print(''.join(f.readlines()))


General metrics. Note TPs and FNs are based on ['Humans']:
true_positive: 5685
true_negative: 4799
false_positive: 555
false_negative: 290
precision_image: 0.9110576923076923
recall_image: 0.9514644351464435
f1_image: 0.9308227589029882
productivity_image: 0.8963391856555846

General metrics on large objects:
large_object_true_positive: 166
large_object_false_negative: 30

Strict metrics on humans:
human_true_positive: 5459
human_true_negative: 5413
human_false_positive: 137
human_false_negative: 516
human_precision_image: 0.975518227305218
human_recall_image: 0.9136401673640168
human_f1_image: 0.9435658110794227
human_productivity_image: 0.9753153153153153

General metrics per operation time and camera pod:
day_safety: 0.9523724580806279
night_safety: 0.9376693766937669
front_safety: nan
rear_safety: 0.9514644351464435
day_front_safety: nan
day_rear_safety: 0.9523724580806279
night_front_safety: nan
night_rear_safety: 0.9376693766937669



In [5]:
out_df = pd.read_csv(output_dir_prod + "/output.csv")

In [6]:
prod_dset = '/data2/jupiter/datasets/halo_rgb_stereo_test_v6_0/'
master_df = pd.read_csv(prod_dset + "master_annotations.csv")

/home/alex.li/miniconda3/envs/cvml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (0,1,22,79,90,91,92,93,94,96,97,99,100,101,102,103,104,105,106,107,117,150,154,155,160,171,174,196,197,199,200,201,202,227,228,235,236,250,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,271,275,276,277,280,284) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
fp_df = out_df[out_df['state'] == 'false_positive']
row = master_df[master_df['id'] == '64dea8faae8b0f37b46e05f7'].iloc[0]
# /mnt/sandbox1/alex.li/results/pmehta_2023_val_bestmodel/halo_rgb_stereo_test_v6_0/human_false_positive/64de93970bf522829d4cd6d8_T13_T15.png

In [8]:
rec_artifacts = rec.get_artifacts(row)
label_converter = LabelConversion(label_map_helper)
rec_label = label_converter.convert_label_for_driveable_terrain(rec_artifacts['label'], json.loads(row['label_map']))


NameError: name 'rec' is not defined

In [ ]:
import os
import random
dataset_path = '/mnt/sandbox1/alex.li/introspection/pmehta_2023_val_bestmodel/halo_rgb_stereo_test_v6_0/'
files = os.listdir(dataset_path + 'false_positive/')
os.mkdir(dataset_path + 'fp_subset_2/')
random.shuffle(files)
for f in files[:50]:
    os.symlink(dataset_path + 'false_positive/' + f, dataset_path + 'fp_subset_2/' + f)

In [ ]:
import os
import random
dataset_path = '/mnt/sandbox1/alex.li/introspection/pmehta_2023_val_bestmodel/halo_humans_on_path_v3/'
files = os.listdir(dataset_path + 'human_false_negative/')
os.mkdir(dataset_path + 'fn_subset/')
random.shuffle(files)
for f in files[:50]:
    os.symlink(dataset_path + 'human_false_negative/' + f, dataset_path + 'fn_subset/' + f)

In [17]:
label_is_cutoff = []
label_is_humanonly = []
label_file = "/home/alex.li/git/JupiterCVML/europa/base/src/europa/dl/config/label_maps/eight_class_train_dust_light_as_sky_birds_as_driveable.csv"
label_map_helper = LabelMapHelper(label_file)
rec = rectifiedrgb.RectifiedRGBNPZ(prod_dset)
deb = debayeredrgb.DebayeredRGBNPZ(prod_dset)
label_converter = LabelConversion(label_map_helper)

for _, row in tqdm(master_df.iterrows(), total=len(master_df)):
    rec_artifacts = rec.get_artifacts(row)
    rec_label = label_converter.convert_label_for_driveable_terrain(rec_artifacts['label'], json.loads(row['label_map']))

    # for id in tqdm(master_df['id']):
    #     row = master_df[master_df['id'] == id].iloc[0]
    # deb_artifacts = deb.get_artifacts(row)
    # deb_label = label_converter.convert_label_for_driveable_terrain(deb_artifacts['label'], json.loads(row['label_map']))
    # rec_human = np.sum(rec_label == 5)
    # center_human = np.sum(rec_label[15:-15,15:-15] == 5)
    # edge_human = rec_human - center_human
    # deb_human = np.sum(deb_label == 5)
    # # Human is on edge only and more present in the debayered image
    # if edge_human > center_human and deb_human > rec_human * 8:
    #     #occluded human
    #     label_is_cutoff.append(True)
    # else:
    #     label_is_cutoff.append(False)
    label_is_humanonly.append(set(np.unique(rec_label)) <= set({5, 255}))

  0%|          | 0/44196 [00:00<?, ?it/s]

100%|██████████| 44196/44196 [30:59<00:00, 23.77it/s]


In [ ]:
sum(label_is_humanonly)

3544

In [86]:
master_df_full_label = master_df[~np.array(label_is_humanonly)]
master_df_full_label.to_csv(prod_dset + "master_annotations_full_label.csv")

In [7]:
master_df_full_label = pd.read_csv(prod_dset + "master_annotations_full_label.csv")

/home/alex.li/miniconda3/envs/cvml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (1,2,91,92,93,94,95,97,98,100,101,102,103,104,105,106,107,108,151,155,156,161,172,175,197,198,200,201,202,203,228,229,236,237,251,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,272,276,277,278,281,285) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
all_uid = master_df['unique_id']
other_uid = master_df_full_label['unique_id']
diff_id = set(all_uid) - set(other_uid)

In [12]:
humanonly_df = master_df[master_df['unique_id'].isin(diff_id)]

In [13]:
len(humanonly_df)

3544

In [27]:
me = pd.read_csv(prod_dset + "master_annotations_human_label.csv")
pawan = pd.read_csv("/home/alex.li/logs/only_humans_labeled_v2.csv")

/home/alex.li/miniconda3/envs/cvml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (23,80,118,200,201,202,203) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/alex.li/miniconda3/envs/cvml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (22,24,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,55,56,57,58,59,60,61,63,64,65,66,67,68,87,88,89,91,93,94,96,97,98,99,100,102,103,123,124,125,127,129,130,132,133,134,135,136,149,152,161) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
me['annotation_pixelwise_0_labeled_objects']

0       [{'data_class': 11, 'vendor_instance_id': 'clo...
1       [{'data_class': 11, 'vendor_instance_id': 'clo...
2       [{'data_class': 11, 'vendor_instance_id': 'clo...
3       [{'data_class': 11, 'vendor_instance_id': 'clo...
4       [{'data_class': 11, 'vendor_instance_id': 'clo...
                              ...                        
3539    [{'data_class': 11, 'vendor_instance_id': 'clo...
3540    [{'data_class': 11, 'vendor_instance_id': 'clo...
3541    [{'data_class': 11, 'vendor_instance_id': 'clo...
3542    [{'data_class': 11, 'vendor_instance_id': 'clo...
3543    [{'data_class': 11, 'vendor_instance_id': 'clo...
Name: annotation_pixelwise_0_labeled_objects, Length: 3544, dtype: object

In [31]:
pawan['id']

0       6533341ce5657b05fa40bfc3
1       653334189c273539da7f7d04
2       653334222138efc004d916c7
3       6533342affd3ef8699ae1e6f
4       65333426eae7767de89823c6
                  ...           
6234    653369cabe3a178a8a6f20f9
6235    653369c8ee836695864544f5
6236    653348e05a561cdcdbb84854
6237    653348e5633f2c5685347e6f
6238    653348ea585e441d76d5b3e8
Name: id, Length: 6239, dtype: object

In [39]:
pawan_uid = pawan['id']
me_uid = me['id']
in_pawan_only = list(set(pawan_uid) - set(me_uid))
in_me_only = list(set(me_uid) - set(pawan_uid))

print(in_pawan_only[:4])
print(in_me_only[:4])

['6533462f7ab15d7ef40013d4', '65334852cc6844cd821bd23e', '653343c15d7de481a25c0bb6', '653346b0ae75c2fba1f524db']
['6533461471334bab28170562', '653376f45d7de481a25c853c', '653356e6bc85c9064f558f76', '653345fe3ea3dbdc70304d42']


In [43]:
print(len(in_me_only))
print(len(in_pawan_only))

30
3599
